In [ ]:
!pip install tavily-python langchain-tavily

In [1]:
import os

os.chdir("../../../")

import asyncio
from textwrap import dedent
from typing import List, Literal, Union

from langchain.tools import BaseTool
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder
from openai import OpenAI
from pydantic import BaseModel, Field

from src.initialization import credential_init

credential_init()

C:\Users\Ling\miniconda3\envs\aicg\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_core.tools import tool

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper

search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search, max_results=1)

@tool
def run_queries(search_queries: list[str]):
    """Run the generated queries."""
    return tavily_tool.batch([{"query": query} for query in search_queries])


In [ ]:
input = ["平民參與救災的風險", "泥石流災害應對安全指導", "如何安全參與災後救援活動"]

result = run_queries({"search_queries":input})

In [ ]:
result

In [4]:
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage
from langchain.memory import ChatMessageHistory
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=6,
    disable_streaming=False
    # other params...
)

In [8]:
tool_calls = response.tool_calls
tool_call = tool_calls[0]

search_results = eval(tool_call['name'])(tool_call['args'])

C:\Users\Ling\AppData\Local\Temp\ipykernel_23516\4236940593.py:4: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  search_results = eval(tool_call['name'])(tool_call['args'])


In [5]:
model_with_tools = model.bind_tools(tools=[run_queries], tool_choice='any')

In [6]:
response = model_with_tools.invoke([HumanMessage(content=["平民參與救災的風險", "泥石流災害應對安全指導", "如何安全參與災後救援活動"])])

In [7]:
response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'run_queries', 'arguments': '{"search_queries": ["\\u5e73\\u6c11\\u53c3\\u8207\\u6ce5\\u77f3\\u6d41\\u6551\\u707d\\u5b89\\u5168\\u6307\\u5c0e", "\\u707d\\u5f8c\\u6551\\u63f4\\u6d3b\\u52d5\\u5b89\\u5168\\u53c3\\u8207"]}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--fe6eb32e-ed3a-4ea3-b468-34e66a4c9181-0', tool_calls=[{'name': 'run_queries', 'args': {'search_queries': ['平民參與泥石流救災安全指導', '災後救援活動安全參與']}, 'id': 'aa04aa46-a66e-44fd-adbc-9dc59155caea', 'type': 'tool_call'}], usage_metadata={'input_tokens': 38, 'output_tokens': 21, 'total_tokens': 59, 'input_token_details': {'cache_read': 0}})

In [9]:
search_results[0]

[{'title': '[PDF] 国家综合减灾“十一五”规划',
  'url': 'https://faolex.fao.org/docs/pdf/chn156592.pdf',
  'content': '动，广泛宣传减灾知识，提高公众安全防范意识和自救互救技能。主办了亚洲减 灾大会等重要的国际减灾会议，签订了《上海合作组织政府间救灾互助协定》。 积极参与国际灾害双边和多边救援行动， 与相关联合国机构和国际、 区域组织建 立了密切的减灾合作关系。 （二）存在的薄弱环节。 目前， 我国的减灾工作还存在一些亟待加强的薄弱环节： 一是一些地方的减 灾综合协调机制尚不健全， 部门间信息共享和协调联动机制、 民间组织等社会力 量参与减灾的机制还不够完善； 二是缺乏减灾综合性法律法规， 相关配套政策不 够完善，灾害保险的作用未得到充分发挥，灾害救助、恢复重建等方面补助标准 偏低；三是灾害监测体系还不够健全，预警信息覆盖面和时效性尚待提高，灾情 监测、采集和评估体系建设滞后；四是防灾减灾基础设施建设有待加强，一些灾 害多发地区的避灾场所建设滞后，大城市和城市群灾害设防水平有待进一步提 高， 农村群众住房防灾抗灾标准普遍较低； 五是基层灾害应急预案体系尚需进一 步健全， 抗灾救灾物资储备体系不够完善， 应急通信、 指挥和交通装备水平落后； 六是减灾资源普查、 灾害风险综合调查评估等方面工作尚未开展， [...] 要和财力可能， 加大对减灾事业的投入， 并按照政府间事权划分纳入各级财政预 算；中央财政适当加大对中西部减灾工作支持力度；适当提高灾害救助标准，完 善救灾补助项目； 加强发挥保险业防灾减灾作用的政策研究和试点工作， 鼓励公 民和企业参加保险， 充分发挥保险对灾害损失的经济补偿和转移分担功能； 广泛 动员社会力量，多渠道筹集减灾资金，健全社会动员机制，加强社会捐助工作， 大力促进慈善事业发展。 （五）加强减灾专业队伍的培育和发展。加强减灾专业人才教育培训体系建 设，提高教育培训能力，开展全方位、多层次的减灾科技教育，提高减灾工作者 整体素质；加大经费、装备投入，提高各级减灾队伍特别是基层队伍的应急救援 能力； 立足减灾工作的实际需要， 整体规划、 统筹协调， 整合优化人才队伍结构， 实现减灾人才队伍和专家队伍的协调发展；构建全民参与减灾的安全文化氛围， 培育和发展社会公

In [11]:
tool_message = ToolMessage(content=[r[0]['content'] for r in search_results], tool_call_id=tool_call['id'])

In [12]:
tool_message

ToolMessage(content=['动，广泛宣传减灾知识，提高公众安全防范意识和自救互救技能。主办了亚洲减 灾大会等重要的国际减灾会议，签订了《上海合作组织政府间救灾互助协定》。 积极参与国际灾害双边和多边救援行动， 与相关联合国机构和国际、 区域组织建 立了密切的减灾合作关系。 （二）存在的薄弱环节。 目前， 我国的减灾工作还存在一些亟待加强的薄弱环节： 一是一些地方的减 灾综合协调机制尚不健全， 部门间信息共享和协调联动机制、 民间组织等社会力 量参与减灾的机制还不够完善； 二是缺乏减灾综合性法律法规， 相关配套政策不 够完善，灾害保险的作用未得到充分发挥，灾害救助、恢复重建等方面补助标准 偏低；三是灾害监测体系还不够健全，预警信息覆盖面和时效性尚待提高，灾情 监测、采集和评估体系建设滞后；四是防灾减灾基础设施建设有待加强，一些灾 害多发地区的避灾场所建设滞后，大城市和城市群灾害设防水平有待进一步提 高， 农村群众住房防灾抗灾标准普遍较低； 五是基层灾害应急预案体系尚需进一 步健全， 抗灾救灾物资储备体系不够完善， 应急通信、 指挥和交通装备水平落后； 六是减灾资源普查、 灾害风险综合调查评估等方面工作尚未开展， [...] 要和财力可能， 加大对减灾事业的投入， 并按照政府间事权划分纳入各级财政预 算；中央财政适当加大对中西部减灾工作支持力度；适当提高灾害救助标准，完 善救灾补助项目； 加强发挥保险业防灾减灾作用的政策研究和试点工作， 鼓励公 民和企业参加保险， 充分发挥保险对灾害损失的经济补偿和转移分担功能； 广泛 动员社会力量，多渠道筹集减灾资金，健全社会动员机制，加强社会捐助工作， 大力促进慈善事业发展。 （五）加强减灾专业队伍的培育和发展。加强减灾专业人才教育培训体系建 设，提高教育培训能力，开展全方位、多层次的减灾科技教育，提高减灾工作者 整体素质；加大经费、装备投入，提高各级减灾队伍特别是基层队伍的应急救援 能力； 立足减灾工作的实际需要， 整体规划、 统筹协调， 整合优化人才队伍结构， 实现减灾人才队伍和专家队伍的协调发展；构建全民参与减灾的安全文化氛围， 培育和发展社会公益组织和志愿者团体，积极参与减灾工作；充分发挥解放军、 武警、公安消防部队、民兵预备役在减灾救灾中的骨干作用。 （六） 推进减灾领域的国际交流与合作。 推动减灾领域信息

In [15]:
model.invoke([HumanMessage(content=["平民參與救災的風險", "泥石流災害應對安全指導", "如何安全參與災後救援活動"]),
                         response,
                         tool_message])

AIMessage(content='平民參與泥石流等災後救援活動，務必以自身安全為首要考量。以下提供安全指導，協助您安全地參與救援：\n\n**一、災前準備與認知：**\n\n*   **了解泥石流特性：** 泥石流具有突發性、流動性、破壞性強等特點。了解這些特性有助於判斷風險。\n*   **關注官方資訊：** 密切關注政府發布的預警信息、疏散指示和災情通報。\n*   **學習急救知識：** 學習基本的急救技能，如心肺復甦術（CPR）、止血、包紮等，以便在緊急情況下幫助他人。\n*   **準備應急物資：** 準備包含飲用水、食物、急救用品、照明設備、通信設備、保暖衣物等的應急包。\n*   **了解救援組織：** 了解當地有哪些救援組織，以及他們的聯繫方式和工作內容。\n\n**二、參與救援的安全原則：**\n\n*   **安全第一：** 永遠將自身安全放在第一位。不要冒不必要的風險。\n*   **服從指揮：** 聽從救援指揮中心的統一指揮，不要擅自行動。\n*   **評估風險：** 在進入災區前，仔細評估周圍環境的風險，如是否有再次發生泥石流的可能、建築物是否穩固、是否有潛在的危險物品等。\n*   **結伴而行：** 不要單獨行動，最好與有經驗的救援人員或志願者結伴而行。\n*   **穿戴防護裝備：** 穿戴必要的防護裝備，如安全帽、手套、防護鞋、口罩等。\n*   **注意衛生：** 災區環境衛生條件差，注意個人衛生，勤洗手，避免感染疾病。\n*   **保持警惕：** 時刻保持警惕，注意觀察周圍環境的變化，如是否有異常聲音、地面是否有裂縫等。\n*   **適時休息：** 救援工作強度大，注意適時休息，補充體力。\n*   **心理調適：** 災後現場可能充滿悲傷和痛苦，注意心理調適，必要時尋求心理輔導。\n\n**三、具體救援活動的安全注意事項：**\n\n*   **搜救：**\n    *   不要進入危險區域，如倒塌的建築物、陡峭的山坡等。\n    *   使用專業的搜救工具，如生命探測儀、搜救犬等。\n    *   注意觀察是否有生命跡象，如呼救聲、敲擊聲等。\n*   **搬運物資：**\n    *   注意搬運姿勢，避免受傷。\n    *   不要搬運過重的物品。\n    *   注意避讓障礙物，防止跌倒。\n*   